# Header

#### We consider three different types of negatives: 
(1) Random: any random passage from the corpus; <br>
(2) BM25: top passages returned by BM25 which don’t contain
the answer but match most question tokens; <br>
(3) Gold: positive passages paired with other questions which appear in the training set. <br>

Regression Objective Function. The cosinesimilarity between the two sentence embeddings
u and v is computed. We use meansquared-error loss as the objective function.

#### 1. How can I construct samples? In DPR they generate negative samples by just sample pairs of format: (one question, answer of another question). <br>
But in my case, with a given context (including several sentences), two questions can map to a similar sentence. <br>
E.g. If the 2 questions are "Ai huong dan NTD?" va "LHL huong dan ai?" then answer_sentence is the same: "Anh LHL huong dan NTD." <br> 
==> Solution:  Pair one question with all (or selected) other sentences of the context that do not contain the answer. <br>

#### 2. Which loss function should I use? <br>
==> Solution: Just use the CosineSimilarityLoss is enough. It's an instance of ContrastiveLoss. <br>
It computes the vectors u = model(input_text[0]) and v = model(input_text[1]) and measures the cosine-similarity between the two. By default, it minimizes the following loss: ||input_label - cos_score_transformation(cosine_sim(u,v))||_2. <br>
Here this loss expects as input two texts and a label of either 0 or 1. If the label == 1, then the distance between the two embeddings is reduced. If the label == 0, then the distance between the embeddings is increased. <br>


In [1]:
%cd "/content/drive/MyDrive/vdt_dsai/dataset"

/content/drive/MyDrive/vdt_dsai/dataset


In [2]:
import json 
with open("train_file.json") as f:
    train_file =  json.load(f)
with open("val_file.json") as f:
    val_file =  json.load(f)
with open("test_file.json") as f:
    test_file =  json.load(f)

In [3]:
def find_sentence(corpus, start_answer):
    '''
    Find sentence contains answer
    * corpus: list sentences of context
    * start_answer: the begin position of answer in context
    '''
    start, end = 0, -1
    sent_idx = -1
    for i in range(len(corpus)):
        sentence = corpus[i]
        start = end + 1
        end = start+len(sentence)
        if start<=start_answer<end:
            sent_idx = i
            break
        elif start_answer == end:
            sent_idx = i + 1
            break 
    return sent_idx

In [4]:
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from rank_bm25 import BM25Okapi
from rank_bm25 import BM25Plus
from copy import deepcopy
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import torch 

def get_topk_sentences_bm25(data, top_k):
    """
    Using sklearn TFIDF to get top k most relevant sentences for each question
    """
    copied_data = deepcopy(data)
    for topic in copied_data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            vectorizer = TfidfVectorizer()
            tokenizer = vectorizer.build_tokenizer()
            tokenized_context = [tokenizer(sentence.lower()) for sentence in context]
            
            # learn statistic of the context sentences.
            bm25 = BM25Plus(tokenized_context)
            for qa in paragraph['qas']:
                question = qa['question']
                query = tokenizer(question)

                # compute scores of each sentence in the context.
                scores = bm25.get_scores(query)
                k = min(top_k, len(context))
                top_results = torch.topk(torch.tensor(scores.reshape(-1)), k=k)
                qa['candidate_indices'] = top_results[1].tolist()
    return copied_data

In [ ]:
!pip install underthesea
!pip install pyvi

In [7]:
import numpy as np
from pyvi.ViTokenizer import tokenize
from underthesea import sent_tokenize


def get_sentence_pairs_bm25(data, topk):
    """
    From data consisting of (context, qas), construct sentence pairs in the format:
    Positive pair: (one question, sentence containing the answer to that question, lable=1.0)
    Negative pair: (one question, sentences in the context of the question that does not contains the answer, label=0.0)
    """
    sentence_pairs = []

    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:

                    # If the answer is from multiple sentences then discard this qa
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1:
                        continue 
                    question = tokenize(qa['question'])

                    # Get the answer sentence. Consider it as positive sentence.
                    answer_start = qa['answers'][0]['answer_start']
                    ans_sent_idx = find_sentence(context, answer_start)
                    pos_sent = tokenize(context[ans_sent_idx])
                    sentence_pairs.append((question, pos_sent, 1.0))

                    # Get all the sentences that are not the answer sentence.
                    neg_sent_idxs = np.delete(np.arange(len(context)), ans_sent_idx)

                    candidate_indices = np.array(qa['candidate_indices'])
                    neg_sent_idxs = candidate_indices[candidate_indices != ans_sent_idx]
                    topk_neg_sent_idxs = neg_sent_idxs[:topk]
                    for neg_idx in topk_neg_sent_idxs:
                        neg_sent = tokenize(context[neg_idx])
                        sentence_pairs.append((question, neg_sent, 0.0))
    
    return sentence_pairs
 

# Sample top 3 negative sentences BM25

In [ ]:
train_file_bm25 = get_topk_sentences_bm25(train_file, 10)
train_pairs = get_sentence_pairs_bm25(train_file_bm25, 3)
train_object = json.dumps(train_pairs, ensure_ascii=False)
with open("train_pairs_bm25.json", "w") as write_file:
    write_file.write(train_object)

val_file_bm25 = get_topk_sentences_bm25(val_file, 10)
val_pairs = get_sentence_pairs_bm25(val_file_bm25, 3)
val_object = json.dumps(val_pairs, ensure_ascii=False)
with open("val_pairs_bm25.json", "w") as write_file:
    write_file.write(val_object)

test_file_bm25 = get_topk_sentences_bm25(test_file, 10)
test_pairs = get_sentence_pairs_bm25(test_file_bm25, 3)
test_object = json.dumps(test_pairs, ensure_ascii=False)
with open("test_pairs_bm25.json", "w") as write_file:
    write_file.write(test_object)

In [ ]:
print("Number of pairs in train pairs:", len(train_pairs))
print("Number of pairs in val pairs:", len(val_pairs))
print("Number of pairs in test pairs:", len(test_pairs))

Number of pairs in train pairs: 58967
Number of pairs in val pairs: 7202
Number of pairs in test pairs: 6367


In [ ]:
with open("train_pairs_bm25.json") as f:
    train_pairs = json.load(f)
train_pairs[:100]

[['Karl_Heinrich Marx trong phiên_âm tiếng Việt_đọc như thế_nào ?',
  'Karl_Heinrich_Marx ( phát_âm tiếng Đức : [ kaːɐ ̯ l ˈhaɪnʀɪç ˈmaːɐ ̯ ks ] , thường được phiên_âm là Các Mác trong các tài_liệu tiếng Việt hoặc Hán Việt là Mã_Khắc_Tư ; sinh 5 tháng 5 năm 1818 tại Trier , Vương_quốc Phổ – mất 14 tháng 3 năm 1883 tại Luân_Đôn , Vương_quốc Anh ) là nhà tư_tưởng người Đức gốc Do thái , và cũng là nhà kinh_tế , nhà lãnh_đạo cách_mạng của Hiệp_hội Người lao_động Quốc_tế .',
  1.0],
 ['Karl_Heinrich Marx trong phiên_âm tiếng Việt_đọc như thế_nào ?',
  'Theo kết_quả nghiên_cứu từ Đại_học Indiana tại Bloomington , Marx là học_giả có ảnh_hưởng nhất thế_giới dựa trên số_lượng nghiên_cứu và độ phổ_biến của các nghiên_cứu .',
  0.0],
 ['Các Mác sinh vào năm nào ?',
  'Karl_Heinrich_Marx ( phát_âm tiếng Đức : [ kaːɐ ̯ l ˈhaɪnʀɪç ˈmaːɐ ̯ ks ] , thường được phiên_âm là Các Mác trong các tài_liệu tiếng Việt hoặc Hán Việt là Mã_Khắc_Tư ; sinh 5 tháng 5 năm 1818 tại Trier , Vương_quốc Phổ – mất 14 thá

# Sample top 5 negative sentences BM25

In [8]:
train_file_bm25 = get_topk_sentences_bm25(train_file, 10)
train_pairs = get_sentence_pairs_bm25(train_file_bm25, 5)
train_object = json.dumps(train_pairs, ensure_ascii=False)
with open("train_pairs_top5.json", "w") as write_file:
    write_file.write(train_object)

val_file_bm25 = get_topk_sentences_bm25(val_file, 10)
val_pairs = get_sentence_pairs_bm25(val_file_bm25, 5)
val_object = json.dumps(val_pairs, ensure_ascii=False)
with open("val_pairs_top5.json", "w") as write_file:
    write_file.write(val_object)

test_file_bm25 = get_topk_sentences_bm25(test_file, 10)
test_pairs = get_sentence_pairs_bm25(test_file_bm25, 5)
test_object = json.dumps(test_pairs, ensure_ascii=False)
with open("test_pairs_top5.json", "w") as write_file:
    write_file.write(test_object)

In [9]:
print("Number of pairs in train pairs:", len(train_pairs))
print("Number of pairs in val pairs:", len(val_pairs))
print("Number of pairs in test pairs:", len(test_pairs))

Number of pairs in train pairs: 75199
Number of pairs in val pairs: 9352
Number of pairs in test pairs: 7974


In [ ]:
with open("train_pairs_top5.json") as f:
    train_pairs = json.load(f)
train_pairs[:100]

[['Karl_Heinrich Marx trong phiên_âm tiếng Việt_đọc như thế_nào ?',
  'Karl_Heinrich_Marx ( phát_âm tiếng Đức : [ kaːɐ ̯ l ˈhaɪnʀɪç ˈmaːɐ ̯ ks ] , thường được phiên_âm là Các Mác trong các tài_liệu tiếng Việt hoặc Hán Việt là Mã_Khắc_Tư ; sinh 5 tháng 5 năm 1818 tại Trier , Vương_quốc Phổ – mất 14 tháng 3 năm 1883 tại Luân_Đôn , Vương_quốc Anh ) là nhà tư_tưởng người Đức gốc Do thái , và cũng là nhà kinh_tế , nhà lãnh_đạo cách_mạng của Hiệp_hội Người lao_động Quốc_tế .',
  1.0],
 ['Karl_Heinrich Marx trong phiên_âm tiếng Việt_đọc như thế_nào ?',
  'Theo kết_quả nghiên_cứu từ Đại_học Indiana tại Bloomington , Marx là học_giả có ảnh_hưởng nhất thế_giới dựa trên số_lượng nghiên_cứu và độ phổ_biến của các nghiên_cứu .',
  0.0],
 ['Các Mác sinh vào năm nào ?',
  'Karl_Heinrich_Marx ( phát_âm tiếng Đức : [ kaːɐ ̯ l ˈhaɪnʀɪç ˈmaːɐ ̯ ks ] , thường được phiên_âm là Các Mác trong các tài_liệu tiếng Việt hoặc Hán Việt là Mã_Khắc_Tư ; sinh 5 tháng 5 năm 1818 tại Trier , Vương_quốc Phổ – mất 14 thá